## Central difference

In [ ]:
using LinearAlgebra

References: 

- Pseudocode in Table 9.1 in Bathe
- https://github.com/ONSAS/ONSAS/blob/master/sources/extractMethodParams.m

---

$$
Mx'' + Cx' + Kx = f(t)
$$

In [1]:
abstract type AbstractAlgorithm end

In [11]:
struct CentralDifference{N} <: AbstractAlgorithm
    δ::N
end

CentralDifference(; δ::N) where N = CentralDifference(δ)

### Setup (Table 9.1, page 771)

In [77]:
function init(M, C, K, U₀, U₀′, U₀′′, alg::CentralDifference)
    Δt = alg.δ

    # compute integration constante
    a₀ = 1/Δt^2
    a₁ = 1/(2*Δt)
    a₂ = 2*a₀
    a₃ = 1/a₂

    U⁻ = U₀ - Δt * U₀′ + a₃ * U₀′′
    M̂ = a₀*M + a₁*C
    
    return a₀, a₁, a₂, U⁻, M̂
end

init (generic function with 1 method)

In [78]:
function solve(ivp::InitialValueProblem{ST, XT},
               alg::CentralDifference{N},
               NSTEPS::Int) where {N, ST<:SecondOrderAffineContinuousSystem, XT}
    
    sys = system(ivp)
    (U₀, U₀′) = initial_state(prob)

    M = mass_matrix(sys)
    C = viscosity_matrix(sys)
    K = stiffness_matrix(sys)
    R = affine_term(sys)
    R = fill(R, NSTEPS)

    U₀′′ = M \ (R[1] - C * U₀′ - K * U₀)
    a₀, a₁, a₂, U⁻, M̂ = init(M, C, K, U₀, U₀′, U₀′′, alg)
    
    U = Vector{Vector{N}}(undef, NSTEPS+1)
    U[1] = U⁻
    U[2] = U₀

    @inbounds for i in 2:NSTEPS
        R̂ᵢ = R[i] - (K - a₂ * M) * U[i] - (a₀*M - a₁*C) * U[i-1]
        U[i+1] = M̂ \ R̂ᵢ
    end
    return U
end

solve (generic function with 2 methods)

In [39]:
x = rand(2)
A = rand(2, 2)
y = A \ x

2-element Array{Float64,1}:
 -0.04858567920853231
  0.4972740054081042

In [40]:
inv(A) * x

2-element Array{Float64,1}:
 -0.04858567920853234
  0.4972740054081041

## Example (9.1 page 773)

In [47]:
using LinearAlgebra
using MathematicalSystems

In [67]:
M = [2 0; 0 1.]
K = [6 -2; -2 4.]
C = zeros(2, 2)
f = [0, 10.]
sys = SecondOrderAffineContinuousSystem(M, C, K, f)

SecondOrderAffineContinuousSystem{Float64,Array{Float64,2},Array{Float64,2},Array{Float64,2},Array{Float64,1}}([2.0 0.0; 0.0 1.0], [0.0 0.0; 0.0 0.0], [6.0 -2.0; -2.0 4.0], [0.0, 10.0])

In [68]:
U₀ = zeros(2)
U₀′ = zeros(2)
alg = CentralDifference(δ=0.28)
U₀′′ = M \ (f - C * U₀′ - K * U₀)

2-element Array{Float64,1}:
  0.0
 10.0

In [69]:
U⁻, M̂  = init(M, C, K, U₀, U₀′, U₀′′, alg)

U⁻

2-element Array{Float64,1}:
 0.0
 0.39200000000000007

In [70]:
M̂

2×2 Array{Float64,2}:
 25.5102   0.0
  0.0     12.7551

In [79]:
prob = InitialValueProblem(sys, (U₀, U₀′))
solve(prob, alg, 5)

6-element Array{Array{Float64,1},1}:
 [0.0, 0.39200000000000007]
 [0.0, 0.0]
 [0.0, 0.39199999999999996]
 [0.0307328, 1.4450688]
 [0.16753063936, 2.83378292736]
 [0.487093853847552, 4.144091532951552]

In [54]:
typeof(prob)

InitialValueProblem{SecondOrderAffineContinuousSystem{Float64,Array{Float64,2},Array{Float64,2},Array{Float64,2},Array{Float64,1}},Tuple{Array{Float64,1},Array{Float64,1}}}

In [ ]:
# sol = solve(sys, alg=CentralDifference(δ=0.28));

In [ ]:
# typeof(sol) == Solution{....}